In [49]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats.stats import pearsonr
import nltk
import pprint, pickle

from keras.datasets import cifar10
from keras.models import *
from keras.layers import *
from keras.layers.core import *
from keras.layers.normalization import *
from keras.optimizers import *
from keras.callbacks import *
from keras import backend as K
from keras.regularizers import *
import theano.tensor as T
import theano

# requires monolingual word aligner
old_path = os.getcwd()
os.chdir('/home/sri/Desktop/monolingual-word-aligner')
from aligner import *
os.chdir(old_path)

In [50]:
train_files = ['MSRpar.txt', 'MSRvid.txt', 'SMTeuroparl.txt']
test_files = ['answer-answer.txt', 'headlines.txt', 'plagiarism.txt', \
              'postediting.txt', 'question-question.txt']

In [51]:
raw_train_data = {}

for t in train_files:
    pkl_file = open("preprocessed_feats/" + t + ".features_pkl", 'rb')

    data = pickle.load(pkl_file)
    raw_train_data[t] = data
    print t, len(data)

MSRpar.txt 750
MSRvid.txt 750
SMTeuroparl.txt 734


In [52]:
MAX_SENTENCE_LEN = 80

In [71]:
from keras.preprocessing import sequence

# process train data for each individual file
train_data = {}
for t in raw_train_data:
    tups = raw_train_data[t]
    
    file_X_train = []
    file_y_train = []
    
    #mlen = 0
    
    i = 0
    for (s1, s2, es1, es2, scr) in tups:
        avg_es1 = np.mean(es1, axis = 1)
        avg_es2 = np.mean(es2, axis = 1)
        feats = np.abs(avg_es1 - avg_es2)
        
        cs = cosine_similarity(avg_es1.reshape(1, -1), avg_es2.reshape(1, -1))
        feats = np.append(feats, cs.flatten())
        
        file_X_train.append(feats)
        file_y_train.append(scr / 5.0)
        
        print "      \r", str(i),
        i += 1
    
    file_X_train = np.array(file_X_train)
    file_y_train = np.array(file_y_train)
       
    train_data[t] = (file_X_train, file_y_train)
    

749


In [72]:
from sklearn.model_selection import train_test_split
val_data = {}

# create train-val split
for t in train_data:
    file_X_train_all, file_y_train_all = train_data[t]
    print file_X_train_all.shape
    
    file_X_train, file_X_val, file_y_train, file_y_val = \
        train_test_split(file_X_train_all, file_y_train_all, test_size = 0.1, random_state = 42)

    train_data[t] = (file_X_train, file_y_train)
    val_data[t] = (file_X_val, file_y_val)

(750, 301)
(734, 301)
(750, 301)


In [73]:
# final training thingy
X_train = None
y_train = None

for t in train_data:
    file_X, file_y = train_data[t]
    print file_X.shape
    
    if (X_train is None):
        X_train = file_X
    else:
        X_train = np.append(X_train, file_X, axis = 0)
        
    if (y_train is None):
        y_train = file_y
    else:
        y_train = np.append(y_train, file_y, axis = 0)

(675, 301)
(660, 301)
(675, 301)


In [74]:
print X_train.shape
print y_train.shape

for v in val_data:
    val_X, val_y = train_data[t]
    print val_X.shape
    print val_y.shape

(2010, 301)
(2010,)
(675, 301)
(675,)
(675, 301)
(675,)
(675, 301)
(675,)


In [78]:
model = Sequential()
model.add(Dense(128, input_dim = X_train.shape[1]))
model.add(PReLU())
model.add(Dropout(0.1))
model.add(Dense(64))
model.add(PReLU())
model.add(Dropout(0.1))
model.add(Dense(32))
model.add(PReLU())
model.add(Dropout(0.1))
model.add(Dense(16))
model.add(PReLU())
model.add(Dropout(0.1))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = Adam(),
      loss = 'mean_absolute_error')
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_38 (Dense)                 (None, 128)           38656       dense_input_12[0][0]             
____________________________________________________________________________________________________
prelu_13 (PReLU)                 (None, 128)           128         dense_38[0][0]                   
____________________________________________________________________________________________________
dropout_27 (Dropout)             (None, 128)           0           prelu_13[0][0]                   
____________________________________________________________________________________________________
dense_39 (Dense)                 (None, 64)            8256        dropout_27[0][0]                 
___________________________________________________________________________________________

In [79]:
model.fit(X_train, y_train, nb_epoch = 50)

Epoch 1/50
2010/2010 [==============================] - 0s - loss: 0.2103     
Epoch 2/50
2010/2010 [==============================] - 0s - loss: 0.1458     
Epoch 3/50
2010/2010 [==============================] - 0s - loss: 0.1346     
Epoch 4/50
2010/2010 [==============================] - 0s - loss: 0.1290     
Epoch 5/50
2010/2010 [==============================] - 0s - loss: 0.1213     
Epoch 6/50
2010/2010 [==============================] - 0s - loss: 0.1164     
Epoch 7/50
2010/2010 [==============================] - 0s - loss: 0.1128     
Epoch 8/50
2010/2010 [==============================] - 0s - loss: 0.1097     
Epoch 9/50
2010/2010 [==============================] - 0s - loss: 0.1084     
Epoch 10/50
2010/2010 [==============================] - 0s - loss: 0.1041     
Epoch 11/50
2010/2010 [==============================] - 0s - loss: 0.1007     
Epoch 12/50
2010/2010 [==============================] - 0s - loss: 0.0998     
Epoch 13/50
2010/2010 [==========================

In [80]:
for v in val_data:
    val_X, val_y = val_data[v]
    
    pred_y = model.predict(val_X).flatten()   
    
    corr, _ = pearsonr(val_y, pred_y)
    print v, corr

MSRpar.txt 0.337958906444
SMTeuroparl.txt 0.692522671614
MSRvid.txt 0.853355443603


In [87]:
for t in test_files:
    pkl_file = open("preprocessed_feats/" + t + ".features_pkl", 'rb')

    tups = pickle.load(pkl_file)
    
    file_X_train = []
    file_y_train = []
    
    i = 0
    for (s1, s2, es1, es2, scr) in tups:
        avg_es1 = np.mean(es1, axis = 1)
        avg_es2 = np.mean(es2, axis = 1)
        feats = np.abs(avg_es1 - avg_es2)
        
        cs = cosine_similarity(avg_es1.reshape(1, -1), avg_es2.reshape(1, -1))
        feats = np.append(feats, cs.flatten())
        
        file_X_train.append(feats)
        file_y_train.append(scr / 5.0)
        
        print "      \r", str(i),
        i += 1
    
    file_X_train = np.array(file_X_train)
    pred_y = model.predict(file_X_train).flatten()
    np.savetxt(t[:-4] + ".predictions", pred_y)

208


In [ ]:
for v in test_data:
    test_X, test_y = test_data[v]
    
    pred_y = model.predict(val_X).flatten()   
    
    corr, _ = pearsonr(val_y, pred_y)
    print v, corr